In [1]:
cd ..

/Users/m.dee/Documents/Data-science-package-beta/data-science-package


<img src="flow_0.png">

In [2]:
from flows.flows import Flows

Welcome to the Data Science Package. First create an object as follows:
For example, use the code below to import the flow 0:
flow = Flows(0)


In [3]:
flow = Flows(0)

Please use the following function to read the data
dataframe_dict = flow.load_data(path: str, files_list: list)
For example: path = './data'
For example: files_list = ['train.csv','test.csv']
The output is a dictionary that contains dataframes e.g.  
dataframe_dict = {'train': train_dataframe,'test': test_dataframe}


In [4]:
path = "./data/flow_0"
files_list = ["train.csv","test.csv"]

In [5]:
dataframe_dict, columns_set = flow.load_data(path, files_list)

The summary of the train dataset:
The train dataset contains 81 columns
There are 43 string categorical columns
There are 18 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
There are 20 columns with numeric continuous values 
There are 0 columns that contain date
There are 0 columns that contain valid nested JSON data
There are 0 columns that contain other type of data
********** End of the summary of the train dataset **********
The summary of the test dataset:
The test dataset contains 80 columns
There are 43 string categorical columns
There are 18 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more 

In [6]:
 columns_set["train"].keys()

dict_keys(['categorical_string', 'categorical_integer', 'continuous', 'date', 'json', 'other'])

In [7]:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict)

The reference dataframe is: train
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in MSZoning feature is: 6
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in Street feature is: 2
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in Alley feature is: 3
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in LotShape feature is: 4
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in LandContour feature is: 4
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in Utilities feature is: 3
the are 2 dat

The summary of the train dataset:
The train dataset contains 81 columns
There are 0 string categorical columns
There are 61 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
There are 20 columns with numeric continuous values 
There are 0 columns that contain date
There are 0 columns that contain valid nested JSON data
There are 0 columns that contain other type of data
********** End of the summary of the train dataset **********
The summary of the test dataset:
The test dataset contains 80 columns
There are 0 string categorical columns
There are 61 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or

In [8]:
ignore_columns = ['id', 'SalePrice']

In [9]:
dataframe_dict, columns_set = flow.scale_data(dataframe_dict, ignore_columns)

The summary of the train dataset:
The train dataset contains 81 columns
There are 0 string categorical columns
There are 61 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
There are 20 columns with numeric continuous values 
There are 0 columns that contain date
There are 0 columns that contain valid nested JSON data
There are 0 columns that contain other type of data
********** End of the summary of the train dataset **********
The summary of the test dataset:
The test dataset contains 80 columns
There are 0 string categorical columns
There are 61 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or

In [10]:
flow.exploring_data(dataframe_dict, "test")

interactive(children=(IntSlider(value=0, description='column_i', max=79), Output()), _dom_classes=('widget-int…

In [11]:
flow.comparing_statistics(dataframe_dict)

interactive(children=(IntSlider(value=0, description='feature_nr', max=80), Output()), _dom_classes=('widget-i…

In [12]:
ignore_columns = ["id", "SalePrice"]
columns = dataframe_dict["train"].columns
train_dataframe = dataframe_dict["train"][[x for x in columns_set["train"]["continuous"] if x not in ignore_columns]]
test_dataframe = dataframe_dict["test"][[x for x in columns_set["train"]["continuous"] if x not in ignore_columns]]
train_target = dataframe_dict["train"]["SalePrice"]

In [13]:
parameters = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "Ridge linear regression",
              "hyperparameters": {"alpha": "optimize",  # alpha:optimize
                                  },
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [14]:
model_index_list, save_models_dir, y_test = flow.training(parameters)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
fold_nr. 1
The quality of the model using the dataset kfold 1
dataset kfold 1: Mean squared error: 1506796185.067
dataset kfold 1:  R2: 78.9 %
fold_nr. 2
The quality of the model using the dataset kfold 2
dataset kfold 2: Mean squared error: 2861589417.8701
dataset kfold 2:  R2: 52.9 %
fold_nr. 3
The quality of the model using the dataset kfold 3
dataset kfold 3: Mean squared error: 1289676591.0382
dataset kfold 3:  R2: 77.9 %
fold_nr. 4
The quality of the model using the dataset kfold 4
dataset kfold 4: Mean squared error: 2065097273.2933
dataset kfold 4:  R2: 69.7 %
fold_nr. 5
The quality of the model using the dataset kfold 5
dataset kfold 5: Mean squared error: 1190322713.9753
dataset kfold 5:  R2: 78.8 %
mean R2 5 Folds:  71.64
mean MSE 5 Folds:  1782696436.2488
The quality of the model using the Evaluating the dataset: train
Evaluating the dataset: train: Mean squared error: 1614587139.4879
Evaluating

In [15]:
parameters_lighgbm = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "lightgbm",
              "hyperparameters": dict(objective='regression', metric='root_mean_squared_error', num_leaves=5,
                                      boost_from_average=True,
                                      learning_rate=0.05, bagging_fraction=0.99, feature_fraction=0.99, max_depth=-1,
                                      num_rounds=10000, min_data_in_leaf=10, boosting='dart')
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [16]:
model_index_list, save_models_dir, y_test = flow.training(parameters_lighgbm)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
fold_nr. 1
The quality of the model using the dataset kfold 1
dataset kfold 1: Mean squared error: 1122052458.1018
dataset kfold 1:  R2: 84.3 %
fold_nr. 2
The quality of the model using the dataset kfold 2
dataset kfold 2: Mean squared error: 1093527367.2384
dataset kfold 2:  R2: 82.0 %
fold_nr. 3
The quality of the model using the dataset kfold 3
dataset kfold 3: Mean squared error: 873025510.1807
dataset kfold 3:  R2: 85.1 %
fold_nr. 4
The quality of the model using the dataset kfold 4
dataset kfold 4: Mean squared error: 1291925866.0344
dataset kfold 4:  R2: 81.1 %
fold_nr. 5
The quality of the model using the dataset kfold 5
dataset kfold 5: Mean squared error: 703399525.7765
dataset kfold 5:  R2: 87.5 %
mean R2 5 Folds:  83.97
mean MSE 5 Folds:  1016786145.4664
The quality of the model using the Evaluating the dataset: train
Evaluating the dataset: train: Mean squared error: 256187947.7154
Evaluating t